<a href="https://colab.research.google.com/github/Hajar-Laktaoui/RecSysExplainability/blob/main/DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import json
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths
train_data_path = "/content/drive/MyDrive/SFTmodel/generated_dataTripAdvisor.json"
val_data_path = "/content/drive/MyDrive/SFTmodel/generated_Val_dataTripAdvisor.json"
huggingface_model_path = "/content/drive/MyDrive/SFTmodel/huggingface_model"

# Step 1: Load Pre-trained GPT-2 Model and Tokenizer
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)  # Move model to GPU
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Save the model and tokenizer to Hugging Face format for consistency
model.save_pretrained(huggingface_model_path)
tokenizer.save_pretrained(huggingface_model_path)

# Step 2: Load Training and Validation Datasets
train_dataset = load_dataset("json", data_files={"train": train_data_path})["train"]
val_dataset = load_dataset("json", data_files={"validation": val_data_path})["validation"]

# Step 3: Tokenize the Dataset
def preprocess_function(examples):
    if "chosen" in examples:
        chosen = tokenizer(
            examples["chosen"], truncation=True, padding="max_length", max_length=512
        )
    else:
        raise ValueError("Dataset examples must include a 'chosen' field.")

    if "rejected" in examples and examples["rejected"]:
        rejected = tokenizer(
            examples["rejected"], truncation=True, padding="max_length", max_length=512
        )
        return {
            "input_ids": chosen["input_ids"],
            "attention_mask": chosen["attention_mask"],
            "rejected_input_ids": rejected["input_ids"],
            "rejected_attention_mask": rejected["attention_mask"],
        }
    else:
        return {
            "input_ids": chosen["input_ids"],
            "attention_mask": chosen["attention_mask"],
        }

# Apply preprocessing to both train and validation datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Step 4: Define Custom DPO Loss Function
def dpo_loss(model, inputs):
    # Move inputs to the GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}

    chosen_outputs = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        labels=inputs["input_ids"]
    )
    if "rejected_input_ids" in inputs:
        rejected_outputs = model(
            input_ids=inputs["rejected_input_ids"],
            attention_mask=inputs["rejected_attention_mask"],
            labels=inputs["rejected_input_ids"]
        )
        return chosen_outputs.loss - rejected_outputs.loss
    else:
        return chosen_outputs.loss

# Step 5: Define Training Arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/SFTmodel/dpo-finetuned-model",  # Save to Google Drive
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=200,
    learning_rate=5e-5,
    weight_decay=0.01,
    report_to="none",
    push_to_hub=False,
    fp16=True,  # Enable mixed precision training for better GPU performance
)

# Step 6: Custom Trainer for DPO
class DPOTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        loss = dpo_loss(model, inputs)
        return (loss, None) if return_outputs else loss

trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
)

# Step 7: Train the Model
trainer.train()

# Step 8: Save the Fine-Tuned Model to Google Drive
output_dir = "/content/drive/MyDrive/SFTmodel/dpo-finetuned-model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Map:   0%|          | 0/256018 [00:00<?, ? examples/s]

Map:   0%|          | 0/32002 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
200,0.107700,No log
400,0.101200,No log
600,0.096000,No log
800,0.098100,No log
1000,0.094900,No log
1200,0.093000,No log
1400,0.092000,No log
1600,0.094700,No log
1800,0.092200,No log
2000,0.093600,No log


Step,Training Loss,Validation Loss
200,0.107700,No log
400,0.101200,No log
600,0.096000,No log
800,0.098100,No log
1000,0.094900,No log
1200,0.093000,No log
1400,0.092000,No log
1600,0.094700,No log
1800,0.092200,No log
2000,0.093600,No log
